In [ ]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image

# Define paths
original_images_path = 'original_images/'
masked_images_path = 'masked_images/'
inpainted_images_path = 'inpainted_images/'

# Ensure the destination folder exists
os.makedirs(inpainted_images_path, exist_ok=True)

# Load a pre-trained inpainting model (Example: DeepFill v2)
# Here we assume you have a function `load_model()` to load your specific inpainting model
# model = load_model() # Placeholder function

# Placeholder for model's inpainting function (this should be defined based on the specific model you use)
def inpaint_image(model, masked_image, mask):
    # Placeholder function for inpainting
    # This function should be defined according to the specifics of your inpainting model
    # For demonstration, we'll assume it returns the inpainted image
    inpainted_image = masked_image  # This should be the actual inpainting process
    return inpainted_image

# Transformation to convert PIL images to tensors
transform = transforms.ToTensor()

# Iterate over masked images
for filename in os.listdir(masked_images_path):
    if filename.lower().endswith(('png', 'jpg', 'jpeg')):
        masked_image_path = os.path.join(masked_images_path, filename)
        original_image_path = os.path.join(original_images_path, filename)

        # Load images
        masked_image = Image.open(masked_image_path).convert('RGB')
        original_image = Image.open(original_image_path).convert('RGB')

        # Convert to tensors
        masked_tensor = transform(masked_image).unsqueeze(0)  # Add batch dimension
        original_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension

        # Create mask tensor (1 for masked pixels, 0 for unmasked)
        mask = (masked_tensor == 0).float()

        # Inpaint the image
        inpainted_tensor = inpaint_image(model, masked_tensor, mask)

        # Convert back to image
        inpainted_image = transforms.ToPILImage()(inpainted_tensor.squeeze(0))

        # Save inpainted image
        inpainted_image.save(os.path.join(inpainted_images_path, filename))

print("Inpainting completed.")


In [1]:
pip install torch torchvision numpy opencv-python requests


Note: you may need to restart the kernel to use updated packages.


In [4]:
!git clone https://github.com/daa233/generative-inpainting-pytorch.git
!cd generative-inpainting-pytorch


Cloning into 'generative-inpainting-pytorch'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 142 (delta 31), reused 22 (delta 22), pack-reused 94
Receiving objects: 100% (142/142), 1.67 MiB | 31.58 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [6]:
# Download model checkpoint (example link, replace with actual if different)
!wget https://example.com/path/to/deepfillv2.pth -O checkpoint.pth


--2024-06-03 13:01:38--  https://example.com/path/to/deepfillv2.pth
Resolving example.com (example.com)... 93.184.215.14, 2606:2800:21f:cb07:6820:80da:af6b:8b2c
Connecting to example.com (example.com)|93.184.215.14|:443... connected.
HTTP request sent, awaiting response... 500 Internal Server Error
2024-06-03 13:01:38 ERROR 500: Internal Server Error.



In [7]:
import os
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from model.networks import Generator
from util.mask import MaskGenerator

# Define paths
original_images_path = '/kaggle/input/fyp1000-128p/'
masked_images_path = '/kaggle/input/fyp1000-128-masked2/'
inpainted_images_path = '/kaggle/working/inpainted'
model_checkpoint_path = '/kaggle/working/checkpoint.pth'

# Ensure the destination folder exists
os.makedirs(inpainted_images_path, exist_ok=True)

# Load the DeepFill v2 model
def load_model(model_checkpoint_path):
    model = Generator()
    model.load_state_dict(torch.load(model_checkpoint_path, map_location=torch.device('cpu')))
    model.eval()  # Set the model to evaluation mode
    return model

# Function to inpaint an image
def inpaint_image(model, masked_image, mask):
    with torch.no_grad():
        # Move tensors to the same device as model
        device = next(model.parameters()).device
        masked_image = masked_image.to(device)
        mask = mask.to(device)
        
        # Concatenate mask and masked image
        input_image = torch.cat((masked_image, mask), dim=1)
        
        # Inpaint the image
        inpainted_image = model(input_image)
        
        # Combine inpainted part with unmasked part
        inpainted_image = inpainted_image * mask + masked_image * (1 - mask)
        
        return inpainted_image

# Initialize the model
model = load_model(model_checkpoint_path)

# Transformation to convert PIL images to tensors
transform = transforms.ToTensor()

# Iterate over masked images
for filename in os.listdir(masked_images_path):
    if filename.lower().endswith(('png', 'jpg', 'jpeg')):
        masked_image_path = os.path.join(masked_images_path, filename)
        original_image_path = os.path.join(original_images_path, filename)

        # Load images
        masked_image = Image.open(masked_image_path).convert('RGB')
        original_image = Image.open(original_image_path).convert('RGB')

        # Convert to tensors
        masked_tensor = transform(masked_image).unsqueeze(0)  # Add batch dimension
        original_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension

        # Create mask tensor (1 for masked pixels, 0 for unmasked)
        mask = (masked_tensor == 0).float()

        # Inpaint the image
        inpainted_tensor = inpaint_image(model, masked_tensor, mask)

        # Convert back to image
        inpainted_image = transforms.ToPILImage()(inpainted_tensor.squeeze(0))

        # Save inpainted image
        inpainted_image.save(os.path.join(inpainted_images_path, filename))

print("Inpainting completed.")


ModuleNotFoundError: No module named 'model'

In [8]:
import os
import torch
import numpy as np
from torchvision import transforms
from PIL import Image

# Clone the repository
!git clone https://github.com/daa233/generative-inpainting-pytorch.git /kaggle/working/generative-inpainting-pytorch

# Download the pre-trained model (example URL, replace with actual if different)
!wget https://example.com/path/to/deepfillv2.pth -O /kaggle/working/checkpoint.pth

# Add the cloned repository to the Python path
import sys
sys.path.append('/kaggle/working/generative-inpainting-pytorch')

# Import necessary modules from the repository
from model.networks import Generator
from util.mask import MaskGenerator

# Define paths
original_images_path = '/kaggle/input/fyp1000-128p/'
masked_images_path = '/kaggle/input/fyp1000-128-masked2/'
inpainted_images_path = '/kaggle/working/inpainted_images/'
model_checkpoint_path = '/kaggle/working/checkpoint.pth'

# Ensure the destination folder exists
os.makedirs(inpainted_images_path, exist_ok=True)

# Load the DeepFill v2 model
def load_model(model_checkpoint_path):
    model = Generator()
    model.load_state_dict(torch.load(model_checkpoint_path, map_location=torch.device('cpu')))
    model.eval()  # Set the model to evaluation mode
    return model

# Function to inpaint an image
def inpaint_image(model, masked_image, mask):
    with torch.no_grad():
        # Move tensors to the same device as model
        device = next(model.parameters()).device
        masked_image = masked_image.to(device)
        mask = mask.to(device)
        
        # Concatenate mask and masked image
        input_image = torch.cat((masked_image, mask), dim=1)
        
        # Inpaint the image
        inpainted_image = model(input_image)
        
        # Combine inpainted part with unmasked part
        inpainted_image = inpainted_image * mask + masked_image * (1 - mask)
        
        return inpainted_image

# Initialize the model
model = load_model(model_checkpoint_path)

# Transformation to convert PIL images to tensors
transform = transforms.ToTensor()

# Iterate over masked images
for filename in os.listdir(masked_images_path):
    if filename.lower().endswith(('png', 'jpg', 'jpeg')):
        masked_image_path = os.path.join(masked_images_path, filename)
        original_image_path = os.path.join(original_images_path, filename)

        # Load images
        masked_image = Image.open(masked_image_path).convert('RGB')
        original_image = Image.open(original_image_path).convert('RGB')

        # Convert to tensors
        masked_tensor = transform(masked_image).unsqueeze(0)  # Add batch dimension
        original_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension

        # Create mask tensor (1 for masked pixels, 0 for unmasked)
        mask = (masked_tensor == 0).float()

        # Inpaint the image
        inpainted_tensor = inpaint_image(model, masked_tensor, mask)

        # Convert back to image
        inpainted_image = transforms.ToPILImage()(inpainted_tensor.squeeze(0))

        # Save inpainted image
        inpainted_image.save(os.path.join(inpainted_images_path, filename))

print("Inpainting completed.")


fatal: destination path '/kaggle/working/generative-inpainting-pytorch' already exists and is not an empty directory.
--2024-06-03 13:10:34--  https://example.com/path/to/deepfillv2.pth
Resolving example.com (example.com)... 93.184.215.14, 2606:2800:21f:cb07:6820:80da:af6b:8b2c
Connecting to example.com (example.com)|93.184.215.14|:443... connected.
HTTP request sent, awaiting response... 500 Internal Server Error
2024-06-03 13:10:34 ERROR 500: Internal Server Error.



ModuleNotFoundError: No module named 'util'

In [1]:
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread

def show_images_horizontally(list_of_files):
    fig = plt.figure(figsize=(12, 3))  # Adjust the figure size as needed
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        ax = fig.add_subplot(1, number_of_files, i + 1)
        image = imread(list_of_files[i])
        plt.imshow(image, cmap='gray')  # You can change the colormap if needed
        plt.axis('off')  # Hide axes
    plt.show()

# Replace 'folder1' and the file paths with your actual folder and image paths
folder1_files = [
    '/kaggle/input/fyp-viewable/15.jpg',
    '/kaggle/input/fyp-viewable/16.jpg',
    '/kaggle/input/fyp-viewable/18.jpg',
    '/kaggle/input/fyp-viewable/30.jpg'
]

show_images_horizontally(folder1_files)


In [2]:
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2  # Import OpenCV for image resizing

def resize_image(image_path, target_size=(512, 512)):
    image = imread(image_path)
    resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
    return resized_image

def show_resized_images_horizontally(list_of_files):
    fig = plt.figure(figsize=(12, 3))  # Adjust the figure size as needed
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        ax = fig.add_subplot(1, number_of_files, i + 1)
        resized_image = resize_image(list_of_files[i])
        plt.imshow(resized_image, cmap='gray')  # You can change the colormap if needed
        plt.axis('off')  # Hide axes
    plt.show()

folder1_files = [
    '/kaggle/input/fyp-viewable/15.jpg',
    '/kaggle/input/fyp-viewable/16.jpg',
    '/kaggle/input/fyp-viewable/18.jpg',
    '/kaggle/input/fyp-viewable/30.jpg'
]

show_resized_images_horizontally(folder1_files)


In [13]:
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2  # Import OpenCV for image resizing

def resize_image(image_path, target_size=(512, 512)):
    image = imread(image_path)
    resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
    return resized_image

def show_resized_images_horizontally(list_of_files):
    fig = plt.figure(figsize=(12, 3))  # Adjust the figure size as needed
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        ax = fig.add_subplot(1, number_of_files, i + 1)
        resized_image = resize_image(list_of_files[i])
        plt.imshow(resized_image, cmap='gray')  # You can change the colormap if needed
        plt.axis('off')  # Hide axes
    plt.show()

folder1_files = [
    '/kaggle/input/fyp1000-128-masked2/6.jpg',
    '/kaggle/input/fyp1000-128-masked2/11.jpg',
    '/kaggle/input/fyp1000-128-masked2/100.jpg',
    '/kaggle/input/fyp1000-128-masked2/992.jpg'
]

show_resized_images_horizontally(folder1_files)
